In [9]:
# %%
# 1. SETUP
import pandas as pd
import numpy as np
import os
import glob
import ast # For parsing the list strings in the crosswalk

# PATHS
DATA_RAW = os.path.join("..", "data", "raw")
DATA_PROCESSED = os.path.join("..", "data", "processed")
os.makedirs(DATA_PROCESSED, exist_ok=True)

In [10]:
# %%
# 2. LOAD RAW TABLES
print("🚀 Loading Raw Data Tables...")

# TABLE 1: ACS Tracts (The Canvas)
t_path = os.path.join(DATA_RAW, "acs_tract_demographics_2023.csv")
if os.path.exists(t_path):
    df_tract_raw = pd.read_csv(t_path, dtype={'state':str, 'county':str, 'tract':str})
    
    # CLEANING: Force Padding and Standard IDs
    df_tract_raw['state'] = df_tract_raw['state'].str.zfill(2)
    df_tract_raw['county'] = df_tract_raw['county'].str.zfill(3)
    df_tract_raw['tract'] = df_tract_raw['tract'].str.split('.').str[0].str.zfill(6)
    
    # Construct Full GEOID (11 Digits)
    df_tract_raw['GEOID'] = df_tract_raw['state'] + df_tract_raw['county'] + df_tract_raw['tract']
    print(f"   ✅ Tracts: {df_tract_raw.shape}")
else:
    print(f"   ❌ Missing: {t_path}")

# TABLE 2: ACS People (The Training Data)
p_files = glob.glob(os.path.join(DATA_RAW, "psam_p*.csv"))
if p_files:
    df_p_raw = pd.concat([pd.read_csv(f) for f in p_files], ignore_index=True)
    print(f"   ✅ People (Combined): {df_p_raw.shape}")
else:
    print(f"   ❌ Missing: psam_p*.csv")

# TABLE 3: SNAP Data (The Anchors)
s_path = os.path.join(DATA_RAW, "snap_county_data_clean.csv")
if os.path.exists(s_path):
    df_snap_raw = pd.read_csv(s_path, dtype={'fips':str})
    
    # --- FIX IS HERE ---
    # Raw FIPS is messy (e.g., '2400101 MD FSP...'). We only want the first 5 digits.
    df_snap_raw['fips'] = df_snap_raw['fips'].astype(str).str.strip().str[:5]
    
    print(f"   ✅ SNAP Data: {df_snap_raw.shape}")
else:
    print(f"   ❌ Missing: {s_path}")

# TABLE 4: Census Relationship File (The Bridge)
r_path = os.path.join(DATA_RAW, "2020_Census_Tract_to_2020_PUMA.csv")
if os.path.exists(r_path):
    df_rel_raw = pd.read_csv(r_path, dtype=str)
    print(f"   ✅ Relationship File: {df_rel_raw.shape}")
else:
    print(f"   ❌ Missing: {r_path}")

🚀 Loading Raw Data Tables...
   ✅ Tracts: (3879, 62)
   ✅ People (Combined): (157694, 287)
   ✅ SNAP Data: (149, 3)
   ✅ Relationship File: (85452, 4)


In [11]:
# %%
# 3. BUILD CROSSWALK (The "Fourth Table" Logic)
print("\n⚙️ Building PUMA-County Crosswalk...")

# 3.1 Prepare Relationship Data
df_rel = df_rel_raw.copy()
rename_map = {
    'STATEFIP': 'STATEFP', 'COUNTYFIP': 'COUNTYFP', 'TRACT': 'TRACTCE', 'PUMA': 'PUMA5CE'
}
df_rel.rename(columns=rename_map, inplace=True)

# Construct GEOID for Join
df_rel['GEOID'] = df_rel['STATEFP'].str.zfill(2) + \
                  df_rel['COUNTYFP'].str.zfill(3) + \
                  df_rel['TRACTCE'].str.zfill(6)

# 3.2 Merge with Tract Population (Needed for Weights)
df_merged = pd.merge(df_rel, df_tract_raw[['GEOID', 'total_population']], on='GEOID', how='inner')

if len(df_merged) == 0:
    print("❌ CRITICAL ERROR: Merge failed. Check GEOID formats.")
else:
    print(f"   -> Merged {len(df_merged)} tract records successfully.")

# 3.3 Aggregate to PUMA Level
crosswalk_rows = []
for puma, group in df_merged.groupby('PUMA5CE'):
    total_puma_pop = group['total_population'].sum()
    unique_counties = group['COUNTYFP'].unique().tolist()
    state_code = group['STATEFP'].iloc[0] # Get State code for this PUMA
    
    county_weights = []
    for county in unique_counties:
        c_pop = group[group['COUNTYFP'] == county]['total_population'].sum()
        w = c_pop / total_puma_pop if total_puma_pop > 0 else 0
        county_weights.append(w)
        
    crosswalk_rows.append({
        'PUMA': puma,
        'STATEFP': state_code,
        'counties': unique_counties,
        'weights': county_weights
    })

df_crosswalk = pd.DataFrame(crosswalk_rows)
print(f"   ✅ Crosswalk Built: {len(df_crosswalk)} PUMAs mapped.")


⚙️ Building PUMA-County Crosswalk...
   -> Merged 3879 tract records successfully.
   ✅ Crosswalk Built: 113 PUMAs mapped.


In [12]:
# %%
# 4. CALCULATE LOCAL SNAP RATES
print("\n🧮 Calculating Weighted SNAP Rates...")

# 4.1 Calculate Raw County SNAP Rates
# Denominator: Sum Tract Population by County (using clean FIPS)
df_tract_raw['GEOID_COUNTY'] = df_tract_raw['state'] + df_tract_raw['county']
county_total_pop = df_tract_raw.groupby('GEOID_COUNTY')['total_population'].sum()

# Numerator: SNAP Counts (indexed by clean 5-digit FIPS)
snap_map = {}
for idx, row in df_snap_raw.iterrows():
    fips = row['fips'] # This is now clean '24001'
    count = row['snap_persons_total']
    
    pop = county_total_pop.get(fips, 0)
    rate = count / pop if pop > 0 else 0
    snap_map[fips] = rate

# 4.2 Calculate Weighted PUMA Rates
puma_rates = []
for idx, row in df_crosswalk.iterrows():
    puma = row['PUMA']
    state = row['STATEFP']
    counties = row['counties'] 
    weights = row['weights']
    
    weighted_rate = 0.0
    for i, cty_code in enumerate(counties):
        full_fips = state + cty_code # '24' + '001' = '24001'
        c_rate = snap_map.get(full_fips, 0)
        weighted_rate += (c_rate * weights[i])
        
    puma_rates.append({'PUMA': puma, 'local_snap_claim_rate': weighted_rate})

df_puma_rates = pd.DataFrame(puma_rates)
print(f"   ✅ Calculated SNAP Rates for {len(df_puma_rates)} PUMAs.")

# Sanity Check
max_rate = df_puma_rates['local_snap_claim_rate'].max()
print(f"   -> Max Rate: {max_rate:.2%}")
if max_rate == 0:
    print("⚠️ WARNING: Rates are still 0. Check FIPS codes again.")


🧮 Calculating Weighted SNAP Rates...
   ✅ Calculated SNAP Rates for 113 PUMAs.
   -> Max Rate: 24.39%


In [13]:
# %%
# 5. ENRICH TRACT DATA
print("\n🛠️ Enriching Tract Data...")
df_tract = df_tract_raw.copy()

# 5.1 Add PUMA ID (Reverse Lookup)
tract_puma_map = df_merged[['GEOID', 'PUMA5CE']].rename(columns={'PUMA5CE': 'PUMA'})
df_tract = df_tract.merge(tract_puma_map, on='GEOID', how='left')

# 5.2 Add Local SNAP Rate
df_tract = df_tract.merge(df_puma_rates, on='PUMA', how='left')

# 5.3 Select Final Columns
age_sex_cols = [c for c in df_tract.columns if c.startswith('m_') or c.startswith('f_')]
race_cols = [c for c in df_tract.columns if c.startswith('race_')]
target_cols = ['GEOID', 'GEOID_COUNTY', 'PUMA', 'local_snap_claim_rate', 
               'total_population', 'poverty_count_est', 'poverty_count_moe'] + \
               age_sex_cols + race_cols

df_tract_final = df_tract[target_cols].copy()
print(f"   ✅ Final Tracts: {df_tract_final.shape}")


🛠️ Enriching Tract Data...
   ✅ Final Tracts: (3879, 60)


In [14]:
# %%
# 6. ENRICH PERSON DATA
print("\n🛠️ Enriching Person Data...")
df_p = df_p_raw.copy()

# 6.1 Clean Target
df_p = df_p.dropna(subset=['POVPIP'])
df_p['is_poor'] = (df_p['POVPIP'] < 100).astype(int)

# 6.2 Rename
rename_map = {
    'PWGTP': 'Person_Weight',
    'AGEP': 'Age',
    'SEX': 'Sex_Code',
    'RAC1P': 'Race_Code',
    'PUMA': 'PUMA'
}
df_p = df_p.rename(columns=rename_map)

# 6.3 Standardize PUMA ID
# PUMS often loads as int (e.g. 101) -> Needs "00101"
df_p['PUMA'] = df_p['PUMA'].astype(str).str.zfill(5)

# 6.4 Add Local SNAP Rate
df_p = df_p.merge(df_puma_rates, on='PUMA', how='left')

# 6.5 Select Final Columns
p_cols = ['is_poor', 'Person_Weight', 'Age', 'Sex_Code', 'Race_Code', 'PUMA', 'local_snap_claim_rate']
df_p_final = df_p[p_cols].copy()

print(f"   ✅ Final People: {df_p_final.shape}")


🛠️ Enriching Person Data...
   ✅ Final People: (150723, 7)


In [15]:
# %%
# 7. SAVE TO DISK
print("\n💾 Saving Processed Datasets...")
df_tract_final.to_csv(os.path.join(DATA_PROCESSED, "df_tract_enriched.csv"), index=False)
df_p_final.to_csv(os.path.join(DATA_PROCESSED, "df_person_enriched.csv"), index=False)
print("✅ Done.")


💾 Saving Processed Datasets...
✅ Done.


In [16]:
# %%
# --- OPTIONAL: CREATE ORACLE DATASET (FOR BENCHMARKING) ---
# This creates a separate "rich" file with Education & Employment
# so we don't break the main pipeline.

import pandas as pd
import os
import glob

# 1. RELOAD RAW DATA (Safe Check)
# If df_p_raw isn't in memory (due to kernel restart), reload it.
if 'df_p_raw' not in locals():
    print("🔄 Reloading Raw Person Data...")
    p_files = glob.glob(os.path.join(DATA_RAW, "psam_p*.csv"))
    if p_files:
        df_p_raw = pd.concat([pd.read_csv(f) for f in p_files], ignore_index=True)
    else:
        raise FileNotFoundError("❌ Could not find psam_p*.csv files.")

# 2. RECOVER CONTEXT (SNAP RATES)
# Instead of re-calculating the crosswalk, we steal the rates from the file we just saved.
# This ensures consistency with the main pipeline.
main_file_path = os.path.join(DATA_PROCESSED, "df_person_enriched.csv")
if os.path.exists(main_file_path):
    # Load just PUMA and SNAP Rate
    df_context = pd.read_csv(main_file_path, usecols=['PUMA', 'local_snap_claim_rate'], dtype={'PUMA':str})
    df_context = df_context.drop_duplicates(subset=['PUMA'])
    print("✅ Recovered SNAP Rates from existing processed data.")
else:
    print("⚠️ Warning: Main processed file missing. Run previous steps first!")
    df_context = pd.DataFrame(columns=['PUMA', 'local_snap_claim_rate'])

# 3. BUILD ORACLE TABLE
print("🛠️ Building Oracle Dataset (Rich Features)...")
df_oracle = df_p_raw.copy()

# A. Clean Target
df_oracle = df_oracle.dropna(subset=['POVPIP'])
df_oracle['is_poor'] = (df_oracle['POVPIP'] < 100).astype(int)

# B. Rename & Select Key Features
# ADDING: SCHL (Education) and ESR (Employment)
rename_map = {
    'PWGTP': 'Person_Weight',
    'AGEP': 'Age',
    'SEX': 'Sex_Code',
    'RAC1P': 'Race_Code',
    'PUMA': 'PUMA',
    'SCHL': 'Education_Code',       # 1-24 scale
    'ESR': 'Employment_Status'      # 1=Employed, 3=Unemployed, 6=Not in Labor Force
}
df_oracle = df_oracle.rename(columns=rename_map)

# C. Standardize PUMA
df_oracle['PUMA'] = df_oracle['PUMA'].astype(str).str.zfill(5)

# D. Merge Context
df_oracle = df_oracle.merge(df_context, on='PUMA', how='left')

# E. Select Final Columns (Rich Version)
oracle_cols = ['is_poor', 'Person_Weight', 'Age', 'Sex_Code', 'Race_Code', 'PUMA', 
               'local_snap_claim_rate', 'Education_Code', 'Employment_Status']

# Filter to ensure columns exist
final_cols = [c for c in oracle_cols if c in df_oracle.columns]
df_oracle_final = df_oracle[final_cols].copy()

# 4. SAVE SEPARATELY
oracle_path = os.path.join(DATA_PROCESSED, "df_person_oracle.csv")
df_oracle_final.to_csv(oracle_path, index=False)

print(f"✅ Saved Oracle Dataset: {df_oracle_final.shape}")
print(f"   path: {oracle_path}")
print(f"   Features: {df_oracle_final.columns.tolist()}")

✅ Recovered SNAP Rates from existing processed data.
🛠️ Building Oracle Dataset (Rich Features)...
✅ Saved Oracle Dataset: (150723, 9)
   path: ../data/processed/df_person_oracle.csv
   Features: ['is_poor', 'Person_Weight', 'Age', 'Sex_Code', 'Race_Code', 'PUMA', 'local_snap_claim_rate', 'Education_Code', 'Employment_Status']
